# Landcover thresholding and binary raster workflow

## Module 1: Isolate the shape of the river, and generate a StudyArea shapefile
Change variables in parts 1-3. Ensure that the Module 1 cell below is selected before you press "Run". Do not press cell-> Run all. There will be consequences, and I cannot guarantee your sanity or safety.

In [2]:
# Welcome to Module 1! 
import arcpy
import os
arcpy.env.overwriteOutput = True

## PART 1: File directories of landsat image and your project folder, change these. 
landsatimage = r"D:\civicsat\landsatcivic-20250908T185205Z-1-001\landsatcivic\l82018.tif" #Direct to your landsat image here
folder= r"C:\Users\jon20\OneDrive\Documents\ArcGIS\Projects\CIVIC" #Direct to your project folder here. Usually it is My Documents\ArcGIS\Projects\*Project name*

## PART 2: Specify landcover value thresholds
watermin = -0.01
watermax = 0.13
landmin = 0.131
landmax = 0.545

## PART 3: Specify the NIR band. It is 5 for for Landsat-8, 4 for Landsat-7
band = 5

# Other variables, don't touch
remapString = f"{watermin} {watermax} 1;{landmin} {landmax} 0"
binaryoutput = os.path.join(folder,"binarytemp.tif") #Put this in a folder, not in a geodatabase
shapefile = os.path.join(folder,"shapefiletemp") 
StudyArea= os.path.join(folder,"StudyArea") #The StudyArea shapefile, showing the full extent of the study area, formerly known as the "blob"

# Extract bands
out_bands_raster = arcpy.ia.ExtractBand(landsatimage, [band])

# Execute Reclassify. Makes a raster where water is 1, land is 0
arcpy.env.addOutputsToMap = False
arcpy.ddd.Reclassify(out_bands_raster, "VALUE", remapString, binaryoutput, "DATA")

# Turn landcover class into vector
arcpy.env.addOutputsToMap = True
arcpy.conversion.RasterToPolygon(binaryoutput, shapefile, "NO_SIMPLIFY", "Value", "SINGLE_OUTER_PART")

# Constructs the StudyArea shapefile based on the vector
arcpy.management.Dissolve(shapefile, StudyArea)

# Adds a gridcode field and value ("0", i.e. land) to the StudyArea shapefile
arcpy.management.AddField(StudyArea, "gridcode", "Long")
with arcpy.da.UpdateCursor(StudyArea, ["gridcode"]) as cursor:
        for row in cursor:
            row[0] = 0
            cursor.updateRow(row)

## Module 2: Generate Final Outputs: 1) Binary raster, and 2) corresponding vector file
Do not run the cell below before first running Module 1

In [3]:
# Welcome to Module 2

## PART 1: Input your isolated river shapefile (IRS)
river= r"C:\Users\jon20\OneDrive\Documents\ArcGIS\Projects\CIVIC\CIVIC.gdb\shapefiletemp_ExportFeatures"

## PART 2: Direct to folder where you want to save your final outputs, a binary raster and its corresponding vector files. Don't put it in a geodatabase.
finaloutput= r"C:\Users\jon20\OneDrive\Documents\ArcGIS\Projects\CIVIC\final"

## PART 3: Name your final outputs. 
binaryvector= "test"
binaryraster= "test.tif" # Don't forget the raster file extension, .tif is preferred for most applications. 

# Other variables, don't touch
temperase= os.path.join(folder, "temperase")
rivermerged= os.path.join(folder, "rivermerged")
bv= r"memory\tempshape"
br= r"memory\br"

# Dissolve river shapefile and assign gridcode of 1
arcpy.env.addOutputsToMap = False
arcpy.management.Dissolve(river, rivermerged)
arcpy.management.AddField(rivermerged, "gridcode", "Long")
with arcpy.da.UpdateCursor(rivermerged, ["gridcode"]) as cursor:
        for row in cursor:
            row[0] = 1
            cursor.updateRow(row)
            
# Construct the binary raster and vector
arcpy.analysis.PairwiseErase(StudyArea, rivermerged, temperase)
arcpy.env.addOutputsToMap = True
arcpy.management.Merge([temperase, rivermerged], bv)
arcpy.conversion.FeatureToRaster(bv, "gridcode", br, landsatimage)


<Result 'memory\\tempshape'>